# Credit Risk Resampling Techniques

In [193]:
import warnings
warnings.filterwarnings('ignore')

In [194]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [195]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]
len(columns)

86

In [196]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.sample(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
26795,15000.0,0.1072,489.10,MORTGAGE,85000.0,Source Verified,Feb-2019,low_risk,n,22.17,...,93.5,0.0,0.0,0.0,365346.0,52627.0,10000.0,63899.0,N,N
29431,30000.0,0.1356,691.23,OWN,100000.0,Verified,Feb-2019,low_risk,n,20.22,...,100.0,0.0,0.0,0.0,275430.0,67436.0,61500.0,70046.0,N,N
68333,12000.0,0.2631,361.50,RENT,50000.0,Not Verified,Jan-2019,low_risk,n,13.13,...,92.3,0.0,0.0,0.0,43400.0,18376.0,27600.0,14300.0,N,N
12347,4000.0,0.0881,126.85,RENT,78680.0,Source Verified,Feb-2019,low_risk,n,13.97,...,48.6,0.0,0.0,0.0,342423.0,351285.0,18400.0,311523.0,N,N
30811,25000.0,0.0881,792.79,MORTGAGE,100000.0,Source Verified,Feb-2019,low_risk,n,23.63,...,100.0,0.0,0.0,0.0,363774.0,47303.0,48100.0,70631.0,N,N
62818,35000.0,0.1447,1204.23,MORTGAGE,79000.0,Source Verified,Jan-2019,high_risk,n,28.04,...,97.5,80.0,0.0,0.0,364816.0,84509.0,28300.0,68183.0,N,N
18367,5000.0,0.2250,192.25,RENT,130000.0,Not Verified,Feb-2019,low_risk,n,10.94,...,95.0,10.0,0.0,0.0,69911.0,30416.0,8900.0,40811.0,N,N
64662,40000.0,0.1033,1296.90,RENT,150000.0,Source Verified,Jan-2019,low_risk,n,13.49,...,100.0,0.0,0.0,0.0,101730.0,61108.0,39800.0,60430.0,N,N
65353,40000.0,0.0819,814.70,MORTGAGE,32000.0,Verified,Jan-2019,low_risk,n,74.27,...,100.0,0.0,0.0,0.0,304530.0,69985.0,114300.0,80730.0,N,N
10097,10000.0,0.0819,203.68,RENT,67500.0,Not Verified,Mar-2019,low_risk,n,15.13,...,100.0,0.0,0.0,0.0,113497.0,26286.0,7000.0,16097.0,N,N


    Month-specific encoding function

In [197]:
# find unique months to add to following dictionary for issue_d encoding and create dictionary to hold for later
def months_num(date_column):
    mo = df[date_column].unique()
    num = [0,1,2]
    months_num = dict(zip(mo,num))
    return(months_num)

    Replace string values with numerical identifiers through Encoding

In [198]:

# realized ther were far too many columns to do this by hand. If I had more time, I would tackle this by creating a loop to iterate over the datatype of each column. if string I could have it auto-encode and join into one row. if datetime I could create an months list like the one above for Q1 and use a lambda to combine new values into new col

# copy df for safety (?)
df2 = df.copy()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# loop that encodes all object (string data) columns
for col in df2:
    if df2[col].dtypes == 'object':
        try:
            month_dict = months_num(col)
            df2[col] = df2[col].apply(lambda x: month_dict[x])
        except KeyError:
            df2[col] = le.fit_transform(df2[col])

df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,0,0,0,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,1,0,0,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,1,0,0,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,1,0,0,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,2,0,0,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


# Split the Data into Training and Testing

In [199]:
# Create our features
X = df2.drop(target, axis=1)


# Create our target
y = df2[target]


In [200]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,1.104015,1.273828,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.917165,0.743862,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,1.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,2.000000,2.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [201]:
# Check the balance of our target values
y['loan_status'].value_counts() # low_risk = 0


0    68470
1      347
Name: loan_status, dtype: int64

In [202]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

    Scale Values

In [203]:
# # Creating the scaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = data_scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [204]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [205]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [206]:
# Calculated the balanced accuracy score

y_pred = model.predict(X_test_scaled)

from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7721189643592354

In [207]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[14768,  2346],
       [   29,    62]])

In [208]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.86      0.68      0.93      0.77      0.60     17114
          1       0.03      0.68      0.86      0.05      0.77      0.58        91

avg / total       0.99      0.86      0.68      0.92      0.77      0.60     17205



### SMOTE Oversampling

In [209]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train
)
Counter(y_resampled)

Counter({'loan_status': 1})

In [210]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [211]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7901470680774174

In [212]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[15197,  1917],
       [   28,    63]])

In [213]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.89      0.69      0.94      0.78      0.63     17114
          1       0.03      0.69      0.89      0.06      0.78      0.60        91

avg / total       0.99      0.89      0.69      0.94      0.78      0.63     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [214]:
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [215]:
# Logistic regression using cluster centroid undersampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [216]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)


0.7660847683343885

In [217]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[13245,  3869],
       [   22,    69]])

In [218]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.77      0.76      0.87      0.77      0.59     17114
          1       0.02      0.76      0.77      0.03      0.77      0.59        91

avg / total       0.99      0.77      0.76      0.87      0.77      0.59     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [219]:
# Use the SMOTEENN technique to perform combination sampling on the data
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [220]:
# Fit a Logistic regression model using random undersampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [221]:
y_pred = model.predict(X_test_scaled)

# Calculate the Balanced Accuracy Score
balanced_accuracy_score(y_test, y_pred)

0.7910254697972356

In [222]:
confusion_matrix(y_test, y_pred)

array([[15039,  2075],
       [   27,    64]])

In [225]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.88      0.70      0.93      0.79      0.63     17114
          1       0.03      0.70      0.88      0.06      0.79      0.61        91

avg / total       0.99      0.88      0.70      0.93      0.79      0.63     17205

